In [ ]:
import pandas as pd
import ujson
import json

In [ ]:

import zstandard
import os
import json
import sys
from datetime import datetime
import logging.handlers

In [ ]:
with open('../data/RS_2016-01.zst','rb') as f:
    data = f.read()


In [ ]:
dctx = zstd.ZstdDecompressor()

In [ ]:
decompressed = dctx.decompress(data)

In [ ]:

log = logging.getLogger("bot")
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler())

def read_lines_zst(file_name):
	with open(file_name, 'rb') as file_handle:
		buffer = ''
		reader = zstandard.ZstdDecompressor(max_window_size=2**31).stream_reader(file_handle)
		while True:
			chunk = reader.read(2**27).decode()
			if not chunk:
				break
			lines = (buffer + chunk).split("\n")

			for line in lines[:-1]:
				yield line, file_handle.tell()

			buffer = lines[-1]
		reader.close()


In [ ]:
file_path = '../data/RS_2016-01.zst'
out_path = '../data/RS_out_test.json'
file_size = os.stat(file_path).st_size
file_lines = 0
file_bytes_processed = 0
created = None
field = "subreddit"
value = "canada"
beginflag = True
bad_lines = 0
try:
    with open(out_path, 'w') as outfile:
        outfile.write('[')
        for line, file_bytes_processed in read_lines_zst(file_path):
            try:
                obj = json.loads(line)
                created = datetime.utcfromtimestamp(int(obj['created_utc']))
                temp = obj[field] == value
                #obj['selftext'] = ''
                #obj['thumbnail'] = ''
                #obj['url'] = ''
                if obj[field] == value:
                    if beginflag:
                        beginflag = False
                    else:
                        outfile.write(',')
                    json.dump(obj, outfile, indent=2)
            except (KeyError, json.JSONDecodeError) as err:
                bad_lines += 1
            file_lines += 1
            if file_lines % 100000 == 0:
                break
                log.info(f"{created.strftime('%Y-%m-%d %H:%M:%S')} : {file_lines:,} : {bad_lines:,} : {(file_bytes_processed / file_size) * 100:.0f}%")
        outfile.write(']')
except Exception as err:
    log.info(err)

log.info(f"Complete : {file_lines:,} : {bad_lines:,}")

In [ ]:
data = pd.read_json('../data/processed_new/RS_2016-01canada.json')
keeped_cols = ['author', 'created_utc', 'title', 'num_comments',
              'score', 'ups', 'domain', 'selftext', 'locked', 'subreddit']
data = data[keeped_cols]
data.created_utc = pd.to_datetime(data.created_utc, unit='s').dt.date
data = data.loc[data.num_comments>=15]
data.head()

In [ ]:
str("212")

In [ ]:
data = pd.read_feather('../data/feather_files/RS_2016canadadf.feather')

In [ ]:
d={}
for x in range(1,10):
    d[x] = data
   # d[x].summary()
for t in d.values():
    print(t.columns)

In [ ]:
d["3"]

In [ ]:
data.extra=0

In [ ]:
data